In [1]:
import pandas as pd, re

def standardize(pid):
    if pd.isna(pid): 
        return None
    s = str(pid).strip()
    return s[:-2] if s.endswith(".0") else s

In [2]:
people = pd.read_csv("mnk_people5.csv", dtype={"mlb_playerID":"string", "npb_playerID":"string", "kbo_playerID":"string"})

if "merge_key" not in people.columns:
    def nm(x): return re.sub(r"[^a-z]", "", str(x).lower())
    people["merge_key"] = (
        people["birthYear"].fillna(-1).astype(int).astype(str).str.zfill(4) +
        people["birthMonth"].fillna(-1).astype(int).astype(str).str.zfill(2) +
        people["birthDay"].fillna(-1).astype(int).astype(str).str.zfill(2) +
        people["nameFirst"].map(nm) + people["nameLast"].map(nm))

id2key = {}
for _, r in people.iterrows():
    if pd.notna(r["mlb_playerID"]):
        id2key[standardize(r["mlb_playerID"])] = r["merge_key"]
    if pd.notna(r["npb_playerID"]):
        id2key[standardize(r["npb_playerID"])] = r["merge_key"]
    if pd.notna(r["kbo_playerID"]):
        id2key[standardize(r["kbo_playerID"])] = r["merge_key"]

In [3]:
def load_and_index(path):
    df = pd.read_csv(path, dtype={"playerID":"string"})
    df["standard_id"] = df["playerID"].map(standardize)
    return {pid: sub.sort_values(["yearID","stint"]).reset_index(drop=True) for pid, sub in df.groupby("standard_id")}

fld_idx = {"MLB": load_and_index("mlb_fielding.csv"),
           "NPB": load_and_index("npb_fielding.csv"),
           "KBO": load_and_index("kbo_fielding.csv")}

meta_cols = ["playerID","yearID","stint","lgID","teamID"]
stat_cols = [c for c in next(iter(fld_idx["MLB"].values())).columns if c not in meta_cols + ["standard_id"]]

In [4]:
tables = []

for _, p in people.iterrows():
    bundles = []

    pid = standardize(p["mlb_playerID"])
    if pid and pid in fld_idx["MLB"]:
        df = fld_idx["MLB"][pid].copy()
        df[["mlb_playerID","npb_playerID","kbo_playerID"]] = [pid,"",""]
        bundles.append(df)

    pid = standardize(p["npb_playerID"])
    if pid and pid in fld_idx["NPB"]:
        df = fld_idx["NPB"][pid].copy()
        df[["mlb_playerID","npb_playerID","kbo_playerID"]] = ["",pid,""]
        bundles.append(df)

    pid = standardize(p["kbo_playerID"])
    if pid and pid in fld_idx["KBO"]:
        df = fld_idx["KBO"][pid].copy()
        df[["mlb_playerID","npb_playerID","kbo_playerID"]] = ["","",pid]
        bundles.append(df)

    if bundles:
        career = (pd.concat(bundles, ignore_index=True).sort_values(["yearID","stint"]).reset_index(drop=True))

        # transition
        trans = [1]
        for i in range(1, len(career)):
            trans.append(trans[-1] + (career.loc[i,"lgID"] != career.loc[i-1,"lgID"]))
        career["transition_indicator"] = trans

        tables.append(career)

In [5]:
if not tables:
    print("ERROR")
else:
    merged = pd.concat(tables, ignore_index=True)

    ordered = ["mlb_playerID","npb_playerID","kbo_playerID","yearID","stint","lgID","teamID","transition_indicator"] + stat_cols
    merged = merged.reindex(columns=ordered, fill_value="")

    merged.to_csv("mnk_fielding5.csv", index=False)
    print(f"{len(merged):,} rows")

226,367 rows
